In [1]:
import pymc as pm
import numpy as np
import arviz as az
from pymc.math import log, exp

%load_ext lab_black
%load_ext watermark

# Deviance

This example demonstrates ...

Adapted from [unit 9: deviances.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/deviances.odc). The lecture has deviances2.odc, but that file wasn't included in the supporting material. We recreate it below anyways.

Todo:
- add more info about DIC and AIC
- add info about WAIC, link to papers showing advantages
    - https://arxiv.org/abs/1507.04544
    - https://arxiv.org/abs/1004.2316

## Associated lecture videos
### Unit 9 Lesson 1

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=88" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

### Unit 9 Lesson 2

In [3]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=89" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

In [4]:
t = np.array((1, 1, 2, 2, 3, 4, 4, 5, 5, 8))

## Model 1

In [5]:
with pm.Model() as m:
    gamma = pm.Gamma("gamma", 0.001, 0.001)
    mu = pm.Normal("mu", 0, tau=0.001)
    beta = mu ** (-1 / gamma)  # convert to PyMC parameterization

    f = pm.Deterministic("f", mu * gamma * t ** (gamma - 1) * exp(-mu * t**gamma))
    loglik = pm.Deterministic("loglik", log(f))
    pm.Deterministic("deviance", -2 * pm.math.sum(loglik))

    pm.Weibull("lik", gamma, beta, observed=t)

    trace = pm.sample(5000, tune=2000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [gamma, mu]


Sampling 4 chains for 2_000 tune and 5_000 draw iterations (8_000 + 20_000 draws total) took 59 seconds.
There were 26 divergences after tuning. Increase `target_accept` or reparameterize.
There were 17 divergences after tuning. Increase `target_accept` or reparameterize.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.


PyMC is not happy with this model, but the results agree with the professor's.

In [6]:
az.summary(trace, var_names=["mu", "gamma", "deviance"], kind="stats")

,mean,sd,hdi_3%,hdi_97%
mu,0.174,0.110,0.020,0.384
gamma,1.491,0.379,0.792,2.197
deviance,43.385,2.397,40.868,47.758


In [7]:
az.waic(trace, scale="deviance")

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/stats.py:1661: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 20000 posterior samples and 10 observations log-likelihood matrix.

              Estimate       SE
deviance_waic    45.01     4.68
p_waic            1.42        -

There has been a warning during the calculation. Please check the results.

## Model 2

In [8]:
with pm.Model() as m2:
    λ = pm.Gamma("λ", 0.001, 0.001)

    f = pm.Deterministic("f", λ * pm.math.exp(-λ * t))
    loglik = pm.Deterministic("loglik", log(f))
    pm.Deterministic("deviance", -2 * pm.math.sum(loglik))

    pm.Exponential("lik", λ, observed=t)

    trace = pm.sample(2000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [λ]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 28 seconds.


In [9]:
az.summary(trace, var_names=["λ", "deviance"], kind="stats")

,mean,sd,hdi_3%,hdi_97%
λ,0.286,0.091,0.123,0.447
deviance,46.061,1.431,45.055,48.654


In [10]:
az.waic(trace, scale="deviance")

Computed from 8000 posterior samples and 10 observations log-likelihood matrix.

              Estimate       SE
deviance_waic    46.51     3.82
p_waic            0.40        -

In [11]:
az.loo(trace)

Computed from 8000 posterior samples and 10 observations log-likelihood matrix.

         Estimate       SE
elpd_loo   -23.26     1.92
p_loo        0.41        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)       10  100.0%
 (0.5, 0.7]   (ok)          0    0.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%

In [12]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

pymc : 4.0.0
numpy: 1.22.4
arviz: 0.12.1

